# P0 with Exception Test Cases

#### Author: Kevin Zhou, Fanping Jiang, Meijing Li

In [ ]:
import nbimporter; nbimporter.options["only_defs"] = False
from IPython.display import display
from P0 import compileString
from ST import printSymTab

def runwasm(wasmfile):
    from IPython.core.display import display, Javascript
    display(Javascript("""
    const params = { 
        P0lib: { 
            write: i => this.append_stream({text: '' + i, name: 'stdout'}),
            writeln: () => this.append_stream({text: '\\n', name: 'stdout'}),
            read: () => window.prompt()
        }
    }

    fetch('""" + wasmfile + """') // asynchronously fetch file, return Response object
      .then(response => response.arrayBuffer()) // read the response to completion and stores it in an ArrayBuffer
      .then(code => WebAssembly.compile(code)) // compile (sharable) code.wasm
      .then(module => WebAssembly.instantiate(module, params)) // create an instance with memory
    // .then(instance => instance.exports.program()); // run the main program; not needed if start function specified
    """))

def runpywasm(wasmfile):
    import pywasm
    def write(s, i): print(i)
    def writeln(s): print('\n')
    def read(s): return int(input())
    vm = pywasm.load(wasmfile, {'P0lib': {'write': write, 'writeln': writeln, 'read': read}})

    
from wasmer import engine, Store, Module, Instance, ImportObject, Function
from wasmer_compiler_cranelift import Compiler

def runwasmer(wasmfile):
    def write(i: int): print(i)
    def writeln(): print('\n')
    def read() -> int: return int(input()) 
    store = Store(engine.JIT(Compiler))
    module = Module(store, open(wasmfile, 'rb').read())
    import_object = ImportObject()
    import_object.register("P0lib", {"write": Function(store, write),
                                     "writeln": Function(store, writeln),"read": Function(store, read)})
    instance = Instance(module, import_object)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

## Tests for Explicit Exceptions

### Square Root Calculator

- Calculate the square root of `-2`

    Since `-2` is smaller than 0, `throw 28` takes affect. The program outputs `123` as expected.

    Run the cell below to see the generated WebAssembly code.

In [ ]:
compileString("""
procedure sqrt(x: integer) → (r: integer)
    if x < 0 then throw 28 else 
        r := 1
        while (r × r) ≤ x do
            r := r + 1
        r := r - 1
program equationsolver
    var a: integer
    try
        a ← sqrt(-2)
        write(a)
    catch 28
        write(123)
    catchall
""")

Then run the cell below to execute the generated WebAssembly code.

In [ ]:
compileString("""
procedure sqrt(x: integer) → (r: integer)
    if x < 0 then throw 28 else 
        r := 1
        while (r × r) ≤ x do
            r := r + 1
        r := r - 1
program equationsolver
    var a: integer
    try
        a ← sqrt(-2)
        write(a)
    catch 28
        write(123)
    catchall
""", 'test.wat')

In [ ]:
!wat2wasm --enable-exceptions test.wat

In [ ]:
runwasm('test.wasm')

- Since the exception tag is up to the user to define, it can be any positive integers, as long as the tag in throw matches the tag in catch.

    Now we change the `throw 28` to `throw 42` as well as `catch 28` to `catch 42`, and run the cells below. You should get the same result.

In [ ]:
compileString("""
procedure sqrt(x: integer) → (r: integer)
    if x < 0 then throw 42 else 
        r := 1
        while (r × r) ≤ x do
            r := r + 1
        r := r - 1
program equationsolver
    var a: integer
    try
        a ← sqrt(-2)
        write(a)
    catch 42
        write(123)
    catchall
""", 'test.wat')

In [ ]:
!wat2wasm --enable-exceptions test.wat

In [ ]:
runwasm('test.wasm')

- You have to match the exception tag in order to catch correctly, otherwise it will not catch as expected.

    Now if we `throw 42` but `catch 20`, there is a mismatch, so it will print nothing.

In [ ]:
compileString("""
procedure sqrt(x: integer) → (r: integer)
    if x < 0 then throw 42 else 
        r := 1
        while (r × r) ≤ x do
            r := r + 1
        r := r - 1
program equationsolver
    var a: integer
    try
        a ← sqrt(-2)
        write(a)
    catch 20
        write(123)
    catchall
""", 'test.wat')

In [ ]:
!wat2wasm --enable-exceptions test.wat

In [ ]:
runwasm('test.wasm')

- Finally, we calculate the square root of `81`

    Since `81` is greater than 0, it outputs the square root of `81` which is `9` as expected.

In [ ]:
compileString("""
procedure sqrt(x: integer) → (r: integer)
    if x < 0 then throw 28 else 
        r := 1
        while (r × r) ≤ x do
            r := r + 1
        r := r - 1
program equationsolver
    var a: integer
    try
        a ← sqrt(81)
        write(a)
    catch 28
        write(123)
    catchall
""", 'test.wat')

In [ ]:
!wat2wasm --enable-exceptions test.wat

In [ ]:
runwasm('test.wasm')

### Multiple Catch Testing

The test cases below is for showing the ability of multiple `catch` application.

The follwing program prints `-1` if it is a negative integer and prints `1` if it is a positive integer. It prints out `0` if the input number is `0`.



Run the cell below to see the generated WebAssembly code.

In [ ]:
compileString("""
procedure posOrNeg(x: integer) → (r: integer)
    if x < 0 then throw 65536
    if x > 0 then throw 65535
    else r := x 
program positiveOrNegative
    var x, a: integer
    try
        x ← read()
        a ← posOrNeg(x)
        write(a)
    catch 65536
        write(-1)
    catch 65535
        write(1)
    catchall
""")

Then run the cell below to execute the generated WebAssembly code.

In [ ]:
compileString("""
procedure posOrNeg(x: integer) → (r: integer)
    if x < 0 then throw 65536
    if x > 0 then throw 65535
    else r := x 
program positiveOrNegative
    var x, a: integer
    try
        x ← read()
        a ← posOrNeg(x)
        write(a)
    catch 65536
        write(-1)
    catch 65535
        write(1)
    catchall
""", 'test.wat')

In [ ]:
!wat2wasm --enable-exceptions test.wat

In [ ]:
runwasm('test.wasm')

## Implicit Exception Test Cases

### Division by 0 

The zero division exception is built-in with the exception tag `111`, the below `P0` code is trying to calculate `99/(34*5-170)`, which is `99/0`. It should trigger `throw 111` thus the statement under `catch 111` will be executed. The output is `111`.


Run the cell below to see the generated WebAssembly code.

In [ ]:
# logic: if a == 0 then throw 111 else b := (99 div a); write(b)
compileString("""
program DividedByZero
    var a, b: integer
        try
            a := (34 × 5 - 170)
            b := (99 div a)
            write(b)
        catch 111
            write(111)
        catchall
""")

Then run the cell below to execute the generated WebAssembly code. The expected output is `111`

In [ ]:
compileString("""
program DividedByZero
    var a, b: integer
        try
            a := (34 × 5 - 170)
            b := (99 div a)
            write(b)
        catch 111
            write(111)
        catchall
""", 'test.wat')

In [ ]:
!wat2wasm --enable-exceptions test.wat

In [ ]:
runwasm("test.wasm")

Now we trying to calculate `99/(34*5-170+3)`, which is `99/3`, and it should output `33` instead of raising exception.

Then run the cell below to execute the generated WebAssembly code.

In [ ]:
compileString("""
program DividedByZero
    var a, b: integer
        try
            a := (34 × 5 - 170 + 3)
            b := (99 div a)
            write(b)
        catch 111
            write(111)
        catchall
""", 'test.wat')

In [ ]:
!wat2wasm --enable-exceptions test.wat

In [ ]:
runwasm("test.wasm")

### Mod by 0

- The same logic goes under the `Mod by 0` exception, with the only difference of the exception tag as `112`.

    The code below is trying to get `99 mod (34*5-170)`, which is `99 mod 0`, it should trigger implicit `throw 112` thus the statement under `catch 112` will be executed. The output is `112`.

Run the cell below to see the generated WebAssembly code.

In [ ]:
# logic: if a == 0 then throw 112 else b := (99 mod a); write(b)
compileString("""
program ModByZero
    var a, b: integer
        try
            a := (34 × 5 - 170)
            b := (99 mod a)
            write(b)
        catch 112
            write(112)
        catchall
""")

Then run the cell below to execute the generated WebAssembly code. The expected output is `112`.

In [ ]:
compileString("""
program ModByZero
    var a, b: integer
        try
            a := (34 × 5 - 170)
            b := (99 mod a)
            write(b)
        catch 112
            write(112)
        catchall
""", 'test.wat')

In [ ]:
!wat2wasm --enable-exceptions test.wat

In [ ]:
runwasm("test.wasm")

Now we trying to calculate the remainder of `99/(34*5-170+5)`, which is `99/5`, and it should output `4` instead of raising exception.

In [ ]:
compileString("""
program ModByZero
    var a, b: integer
        try
            a := (34 × 5 - 170 + 5)
            b := (99 mod a)
            write(b)
        catch 112
            write(112)
        catchall
""", 'test.wat')

In [ ]:
!wat2wasm --enable-exceptions test.wat

In [ ]:
runwasm("test.wasm")

### Index Out of Bounds Exception

The `index out of bounds` exception is built-in with the exception tag of `110`. Below is an example.


Since the program is trying to access the index that is out of the bound, `throw 110` will be triggered and thus the statement under `catch 110` will be executed, therefore the output is `110`.

Run the cell below to see the generated WebAssembly code.

In [ ]:
# logic: if (i < x.tp.lower) or (i >= x.tp.lower + x.tp.length) then throw else write(x[i])
compileString("""
var x: [2 .. 4] → integer
program IndexOutOfBounds
    var i: integer
    x[2] := 2; x[3] := 3; x[4] := 4
    try
        i := 5
        write(x[i])
    catch 110
        write(110)
    catchall
""")

Then run the cell below to execute the generated WebAssembly code.

In [ ]:
compileString("""
var x: [2 .. 4] → integer
program IndexOutOfBounds
    var i: integer
    x[2] := 2; x[3] := 3; x[4] := 4
    try
        i := 5
        write(x[i])
    catch 110
        write(110)
    catchall
""", 'test.wat')

In [ ]:
!wat2wasm --enable-exceptions test.wat

In [ ]:
runwasm("test.wasm")

What if the index is negative? The negative index is also out of bound, thus the output is still `110`.

In [ ]:
compileString("""
var x: [2 .. 4] → integer
program IndexOutOfBounds
    var i: integer
    x[2] := 2; x[3] := 3; x[4] := 4
    try
        i := -5
        write(x[i])
    catch 110
        write(110)
    catchall
""", 'test.wat')

In [ ]:
!wat2wasm --enable-exceptions test.wat

In [ ]:
runwasm("test.wasm")

If the index is valid, the output is `x[i]`

In [ ]:
compileString("""
var x: [2 .. 4] → integer
program IndexOutOfBounds
    var i: integer
    x[2] := 2; x[3] := 3; x[4] := 4
    try
        i := 2
        write(x[i])
    catch 110
        write(110)
    catchall
""", 'test.wat')

In [ ]:
!wat2wasm --enable-exceptions test.wat

In [ ]:
runwasm("test.wasm")